In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Canavan_Disease/GSE41445'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Basal gene expression data of 21 cell lines (18 cancer and 3 non-tumorigenic)"
!Series_summary	"We profiled basal gene expressed levels of 21 cell lines (18 cancer and 3 non-tumorigenic) using Affymetrix HG-U133_plus2 GeneChip microarrays."
!Series_summary	"Goal of the experiment was to benchmark a number of algorithms for biomarker detection all of which utilize gene expression data."
!Series_overall_design	"Cell lines were cultivated in appropriate media according to supplier recommendations. Goal was to acquire a basal gene expression profile of 21 common cell lines."
Sample Characteristics Dictionary:
{0: ['gender: male', 'gender: female'], 1: ['organ: Kidney', 'organ: Lung', 'organ: Colon', 'organ: Prostate', 'organ: Skin', 'organ: Cervix', 'organ: Breast', 'organ: Pancreas', 'organ: Ovary', 'organ: Bone'], 2: ['disease: renal cell adenocarcinoma', 'disease: non-small cell carcinoma', 'disease: colorectal adenocarcinoma', 'disease: moderately

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # since Affymetrix HG-U133_plus2 GeneChip microarrays represent gene expression data

# Variable Availability and Data Type Conversion
# Identifying the keys for the required variables based on sample characteristics

# Canavan_Disease (looking for evidence in sample characteristics)
for key, values in {2: ['disease: non-tumorigenic; aspartoacylase deficiency; possible Canavan disease']}.items():
    if any('Canavan disease' in value for value in values):
        trait_row = key
        break

# Age (not available based on given data)
age_row = None  

# Gender data availability
for key, values in {0: ['gender: male', 'gender: female']}.items():
    if any('gender' in value for value in values):
        gender_row = key
        break

# Data Type Conversion Functions
# Canavan_Disease
def convert_trait(value):
    value = value.split(':')[1].strip().lower()
    trait_dict = {'non-tumorigenic; aspartoacylase deficiency; possible canavan disease': 1}
    return trait_dict.get(value, 0)

# Age (since age data is not available, conversion function is not needed)
convert_age = None

# Gender
def convert_gender(value):
    value = value.split(':')[1].strip().lower()
    if value == 'male':
        return 1
    elif value == 'female':
        return 0
    else:
        return None

save_cohort_info('GSE41445', './preprocessed/Canavan_Disease/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Canavan_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Canavan_Disease/trait_data/GSE41445.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1017454': [0, 1], 'GSM1017455': [0, 1], 'GSM1017456': [0, 1], 'GSM1017457': [0, 1], 'GSM1017458': [0, 1], 'GSM1017459': [0, 1], 'GSM1017460': [0, 1], 'GSM1017461': [0, 1], 'GSM1017462': [0, 1], 'GSM1017463': [0, 1], 'GSM1017464': [0, 1], 'GSM1017465': [0, 1], 'GSM1017466': [0, 0], 'GSM1017467': [0, 0], 'GSM1017468': [0, 0], 'GSM1017469': [0, 1], 'GSM1017470': [0, 1], 'GSM1017471': [0, 1], 'GSM1017472': [0, 0], 'GSM1017473': [0, 0], 'GSM1017474': [0, 0], 'GSM1017475': [1, 1], 'GSM1017476': [1, 1], 'GSM1017477': [1, 1], 'GSM1017478': [0, 0], 'GSM1017479': [0, 0], 'GSM1017480': [0, 0], 'GSM1017481': [0, 0], 'GSM1017482': [0, 0], 'GSM1017483': [0, 0], 'GSM1017484': [0, 0], 'GSM1017485': [0, 0], 'GSM1017486': [0, 0], 'GSM1017487': [0, 0], 'GSM1017488': [0, 0], 'GSM1017489': [0, 0], 'GSM1017490': [0, 0], 'GSM1017491': [0, 0], 'GSM1017492': [0, 0], 'GSM1017493': [0, 0], 'GSM1017494': [0, 0], 'GSM1017495': [0, 0], 'GSM1017496': [0, 1], 'GSM1017497': [0, 1], 'GSM1017498': [0, 1], 'GSM10174

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the key names for identifiers and gene symbols from the annotation data.
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Canavan_Disease/gene_data/GSE41445.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Canavan_Disease')

# Define trait_biased
trait_biased = trait_biased

# 4. Save the cohort information.
save_cohort_info('GSE41445', './preprocessed/Canavan_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Canavan_Disease/GSE41445.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Canavan_Disease', the least common label is '1.0' with 3 occurrences. This represents 4.76% of the dataset.
The distribution of the feature 'Canavan_Disease' in this dataset is severely biased.

For the feature 'Gender', the least common label is '1.0' with 27 occurrences. This represents 42.86% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Canavan_Disease/cohort_info.json
